In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Initial Exploration & Loading
# تحميل البيانات كما هو مطلوب في التمرين
df = sns.load_dataset('titanic')

# عرض معلومات أولية
print("Dataset Info:")
print(df.info())

# 2. Data Cleaning
# التعامل مع القيم المفقودة
# ملء العمر بالوسيط (Median)
df['age'] = df['age'].fillna(df['age'].median())
# ملء ميناء الانطلاق بالمنوال (Mode)
df['embarked'] = df['embarked'].fillna(df['embarked'].mode()[0])

# حذف الأعمدة غير الضرورية أو المكررة كما هو مقترح (drop irrelevant columns)
# ملاحظة: deck يحتوي على الكثير من القيم المفقودة، و alive مكرر لـ survived
cols_to_drop = ['deck', 'embark_town', 'alive', 'who', 'adult_male', 'class']
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

# حذف التكرارات
df = df.drop_duplicates()

# 3. Feature Engineering
# إنشاء ميزة حجم العائلة: FamilySize = sibsp + parch + 1
df['FamilySize'] = df['sibsp'] + df['parch'] + 1
# إنشاء ميزة "وحيد": IsAlone
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)

# ملاحظة: استخراج الألقاب (Titles) من الاسم غير ممكن مباشرة هنا لأن مجموعة بيانات seaborn 
# عادة لا تحتوي على عمود 'name'. إذا كانت متوفرة، يتم استخدام df['name'].str.extract.

# 4. Categorical Encoding
# تحويل الجنس (sex) إلى أرقام (Label Encoding)
le = LabelEncoder()
df['sex'] = le.fit_transform(df['sex'])

# تحويل embarked إلى One-Hot Encoding
df = pd.get_dummies(df, columns=['embarked'], prefix='embarked', drop_first=True)

# 5. Outlier Handling (Example)
# التعامل مع القيم المتطرفة في السعر (Fare)
# تحديد عتبة 99% واستبدال القيم الأكبر بها
q99 = df['fare'].quantile(0.99)
df.loc[df['fare'] > q99, 'fare'] = q99

# 6. Scaling / Normalization
# توحيد مقاييس الأعمدة الرقمية
scaler = StandardScaler()
num_cols = ['age', 'fare', 'FamilySize']
df[num_cols] = scaler.fit_transform(df[num_cols])

# 7. Train/Test Split
# تقسيم البيانات: 80% تدريب و 20% اختبار
X = df.drop('survived', axis=1)
y = df['survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 8. Modeling
# Model 1: Logistic Regression
lr = LogisticRegression(max_iter=500, random_state=42)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

# Model 2: K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

# 9. Evaluation
print("\n--- Logistic Regression Results ---")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("\nClassification Report:\n", classification_report(y_test, y_pred_lr))

print("\n--- KNN Results ---")
print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("\nClassification Report:\n", classification_report(y_test, y_pred_knn))


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB
None

--- Logistic Regression Results ---
Accuracy: 0.77419354838